# Census - Median Household Income (MHI) and New York Times Mask Data

#### This notebook contains code for gathering Census data tables and converting them to dataframes. 
#### Please use caution when altering this code.

In [2]:
import pandas as pd
from census import Census #<-- Python wrapper for census API
import requests
import os
import matplotlib.pyplot as plt
import matplotlib

# Census API Key
from config import api_key

# provide the api key and the year to establish a session
c = Census(api_key, year=2018)

# Set an option to allow up to 300 characters to print in each column
pd.set_option('max_colwidth', 300)

In [3]:
tables = c.acs5.tables()

# The tables variable contains a list of dicts, so we can convert directly to a dataframe
table_df = pd.DataFrame(tables)

##### The cell below gathering columns names from the Census Total Population table that only contain estimate and are int data types.
##### Those table names are added to a string variable and then pulled from the census data.
##### Dictionary key names are replaced withe more meaningful names.
##### Data is saved to a dataframe and exported as a csv file.

In [8]:
table_id = 'B19013'      

# Capture the variables URL from the table_df
url = table_df.loc[table_df['name']==table_id, 'variables'].values[0]

# Make the API call
response = requests.get(url).json()

# convert the response to a DataFrame
variables = pd.DataFrame(response['variables']).transpose()

print(f"Number of available variables: {len(variables)}")

table_info = variables[(variables['predicateType']=='int') & (variables['label'].str.contains("Estimate"))]   


table_columns = ""
for ind in table_info.index:       
    table_columns = table_columns + f"{ind},"
    
    
column_list = table_columns[:-1]

census_data = c.acs5.get(("NAME", column_list), 
                          {'for': 'county:*'})

for i in range(len(table_info)) : 
    for d in census_data:
        d[table_info.iloc[i, 0]] = d.pop(table_info.index[i]) 
    

census_B19013_MHI_df = pd.DataFrame(census_data)
census_B19013_MHI_df.to_csv("census_B19013_MHI.csv", encoding="utf-8", index=False )
census_B19013_MHI_df


Number of available variables: 4


,NAME,state,county,Estimate!!Median household income in the past 12 months (in 2018 inflation-adjusted dollars)
0,"Washington County, Mississippi",28,151,30834.0
1,"Perry County, Mississippi",28,111,39007.0
2,"Choctaw County, Mississippi",28,019,37203.0
3,"Itawamba County, Mississippi",28,057,40510.0
4,"Carroll County, Mississippi",28,015,43060.0
...,...,...,...,...
3215,"Clayton County, Iowa",19,043,52828.0
3216,"Buena Vista County, Iowa",19,021,54556.0
3217,"Guthrie County, Iowa",19,077,57075.0
3218,"Humboldt County, Iowa",19,091,52219.0


### Census MHI Column Changes

In [11]:
census_B19013_MHI_df.dtypes

census_B19013_MHI_df['COUNTYFP'] = census_B19013_MHI_df['state'] + census_B19013_MHI_df['county']

census_MHI_sorted = census_B19013_MHI_df.sort_values(by=['COUNTYFP'])
census_MHI_sorted = census_MHI_sorted.reset_index(drop=True)

census_MHI_sorted['State Name'] = ''
census_MHI_sorted['County Name'] = ''


for index, row in census_MHI_sorted.iterrows():
    name_all = row['NAME']
    name_list = name_all.split(',')
    clean_state = name_list[1][1:]
    census_MHI_sorted.loc[index, 'State Name']= clean_state
    census_MHI_sorted.loc[index, 'County Name']= name_list[0]
    
census_MHI_sorted = census_MHI_sorted.rename(columns={"Estimate!!Median household income in the past 12 months (in 2018 inflation-adjusted dollars)": "Median Income"})
census_MHI_sorted

,NAME,state,county,Median Income,COUNTYFP,State Name,County Name
0,"Autauga County, Alabama",01,001,58786.0,01001,Alabama,Autauga County
1,"Baldwin County, Alabama",01,003,55962.0,01003,Alabama,Baldwin County
2,"Barbour County, Alabama",01,005,34186.0,01005,Alabama,Barbour County
3,"Bibb County, Alabama",01,007,45340.0,01007,Alabama,Bibb County
4,"Blount County, Alabama",01,009,48695.0,01009,Alabama,Blount County
...,...,...,...,...,...,...,...
3215,"Vega Baja Municipio, Puerto Rico",72,145,19096.0,72145,Puerto Rico,Vega Baja Municipio
3216,"Vieques Municipio, Puerto Rico",72,147,15539.0,72147,Puerto Rico,Vieques Municipio
3217,"Villalba Municipio, Puerto Rico",72,149,19855.0,72149,Puerto Rico,Villalba Municipio
3218,"Yabucoa Municipio, Puerto Rico",72,151,16013.0,72151,Puerto Rico,Yabucoa Municipio


In [13]:
infile = os.path.join("..", "NY_Times_Data", "mask-use-by-county.csv" )
nytimes_mask_data = pd.read_csv(infile)
# nytimes_mask_data.dtypes
nytimes_mask_data['COUNTYFP'] = nytimes_mask_data['COUNTYFP'].astype(str)
nytimes_mask_data['COUNTYFP'] = nytimes_mask_data['COUNTYFP'].str.zfill(5)
nytimes_mask_data

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,01001,0.053,0.074,0.134,0.295,0.444
1,01003,0.083,0.059,0.098,0.323,0.436
2,01005,0.067,0.121,0.120,0.201,0.491
3,01007,0.020,0.034,0.096,0.278,0.572
4,01009,0.053,0.114,0.180,0.194,0.459
...,...,...,...,...,...,...
3137,56037,0.061,0.295,0.230,0.146,0.268
3138,56039,0.095,0.157,0.160,0.247,0.340
3139,56041,0.098,0.278,0.154,0.207,0.264
3140,56043,0.204,0.155,0.069,0.285,0.287


In [14]:
nytimesmask_censusMHI = pd.merge(nytimes_mask_data, census_MHI_sorted, on="COUNTYFP", how="left")
nytimesmask_censusMHI

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,NAME,state,county,Median Income,State Name,County Name
0,01001,0.053,0.074,0.134,0.295,0.444,"Autauga County, Alabama",01,001,58786.0,Alabama,Autauga County
1,01003,0.083,0.059,0.098,0.323,0.436,"Baldwin County, Alabama",01,003,55962.0,Alabama,Baldwin County
2,01005,0.067,0.121,0.120,0.201,0.491,"Barbour County, Alabama",01,005,34186.0,Alabama,Barbour County
3,01007,0.020,0.034,0.096,0.278,0.572,"Bibb County, Alabama",01,007,45340.0,Alabama,Bibb County
4,01009,0.053,0.114,0.180,0.194,0.459,"Blount County, Alabama",01,009,48695.0,Alabama,Blount County
...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,0.061,0.295,0.230,0.146,0.268,"Sweetwater County, Wyoming",56,037,73008.0,Wyoming,Sweetwater County
3138,56039,0.095,0.157,0.160,0.247,0.340,"Teton County, Wyoming",56,039,83831.0,Wyoming,Teton County
3139,56041,0.098,0.278,0.154,0.207,0.264,"Uinta County, Wyoming",56,041,58235.0,Wyoming,Uinta County
3140,56043,0.204,0.155,0.069,0.285,0.287,"Washakie County, Wyoming",56,043,53426.0,Wyoming,Washakie County
